In [1]:
import json
import torch
import random
from pathlib import Path
from tqdm import tqdm
import numpy as np

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
NUM_EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 5e-5

print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

In [2]:
train_input_file = Path("dataset/TL/용례_게임tl.json")
val_input_file = Path("dataset/VL/용례_게임vl.json")

In [3]:
train_processed_data = []
labels = set(['O'])

print("JSON 파일 로딩 중...")
with open(train_input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

total_samples = len(data)
print(f"총 {total_samples:,}개 샘플 발견")

for example in tqdm(data, desc="Processing training examples"):
    sentence = example.get('sentence', '')
    tokens = example.get('tokens', [])
    
    if sentence and tokens:
        char_tags = ['O'] * len(sentence)
        
        for token in tokens:
            start = token['start']
            length = token['length']
            facet = token.get('facet', 'TERM')
            
            if start < len(sentence):
                char_tags[start] = f'B-{facet}'
            
            for i in range(start + 1, start + length):
                if i < len(sentence):
                    char_tags[i] = f'I-{facet}'
        
        chars = list(sentence)
        tags = char_tags
        labels.update(tags)
        
        train_processed_data.append({
            'id': example.get('id'),
            'sentence': sentence,
            'chars': chars,
            'tags': tags,
            'tokens': tokens
        })

# 메모리 정리
del data

print(f"학습 데이터: {len(train_processed_data):,}개 예제 처리 완료")

JSON 파일 로딩 중...
총 199,504개 샘플 발견


Processing training examples: 100%|█████████████████████████████████████████| 199504/199504 [00:02<00:00, 73609.44it/s]

학습 데이터: 199,504개 예제 처리 완료


In [4]:
if val_input_file and val_input_file.exists():
    print(f"\n검증 데이터 처리 중: {val_input_file}")
    val_processed_data = []
    
    print("JSON 파일 로딩 중...")
    with open(val_input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    total_samples = len(data)
    print(f"총 {total_samples:,}개 샘플 발견")
    
    for example in tqdm(data, desc="Processing validation examples"):
        sentence = example.get('sentence', '')
        tokens = example.get('tokens', [])
        
        if sentence and tokens:
            char_tags = ['O'] * len(sentence)
            
            for token in tokens:
                start = token['start']
                length = token['length']
                facet = token.get('facet', 'TERM')
                
                if start < len(sentence):
                    char_tags[start] = f'B-{facet}'
                
                for i in range(start + 1, start + length):
                    if i < len(sentence):
                        char_tags[i] = f'I-{facet}'
            
            chars = list(sentence)
            tags = char_tags
            labels.update(tags)
            
            val_processed_data.append({
                'id': example.get('id'),
                'sentence': sentence,
                'chars': chars,
                'tags': tags,
                'tokens': tokens
            })
    
    del data
    
    print(f"검증 데이터: {len(val_processed_data):,}개 예제 처리 완료")
    
    random.shuffle(train_processed_data)
    train_size = int(len(train_processed_data) * 0.9)
    
    train_data = train_processed_data[:train_size]
    test_data = train_processed_data[train_size:]
    val_data = val_processed_data
    
else:
    print("\n검증 파일이 없어 학습 데이터를 분할합니다.")
    random.shuffle(train_processed_data)
    total = len(train_processed_data)
    train_size = int(total * 0.8)
    val_size = int(total * 0.1)
    
    train_data = train_processed_data[:train_size]
    val_data = train_processed_data[train_size:train_size + val_size]
    test_data = train_processed_data[train_size + val_size:]

print(f"\n데이터 분할 완료:")
print(f"  - Train: {len(train_data):,} samples")
print(f"  - Val: {len(val_data):,} samples")
print(f"  - Test: {len(test_data):,} samples")


검증 데이터 처리 중: dataset\VL\용례_게임vl.json
JSON 파일 로딩 중...
총 24,938개 샘플 발견


Processing validation examples: 100%|████████████████████████████████████████| 24938/24938 [00:00<00:00, 110602.14it/s]


검증 데이터: 24,938개 예제 처리 완료

데이터 분할 완료:
  - Train: 179,553 samples
  - Val: 24,938 samples
  - Test: 19,951 samples


In [5]:
with open("processed/train.json", 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)
with open("processed/val.json", 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)
with open("processed/test.json", 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

label2id = {label: idx for idx, label in enumerate(sorted(labels))}
id2label = {idx: label for label, idx in label2id.items()}

label_map = {
    'label2id': label2id,
    'id2label': id2label,
    'num_labels': len(labels)
}

with open("processed/label_map.json", 'w', encoding='utf-8') as f:
    json.dump(label_map, f, ensure_ascii=False, indent=2)

print("전처리 완료! 데이터 저장됨")

전처리 완료! 데이터 저장됨


In [6]:
with open("processed/label_map.json", 'r', encoding='utf-8') as f:
    label_map = json.load(f)

label2id = label_map['label2id']
id2label = {int(k): v for k, v in label_map['id2label'].items()}
num_labels = label_map['num_labels']

print(f"레이블 수: {num_labels}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    use_safetensors=True
)

print("모델 초기화 완료!")

레이블 수: 39


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 초기화 완료!


In [7]:
print("데이터 로딩 중...")
with open("processed/train.json", 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open("processed/val.json", 'r', encoding='utf-8') as f:
    val_data = json.load(f)
with open("processed/test.json", 'r', encoding='utf-8') as f:
    test_data = json.load(f)

print(f"  - Train: {len(train_data)} samples")
print(f"  - Val: {len(val_data)} samples")
print(f"  - Test: {len(test_data)} samples")

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

print("\n토큰화 중...")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['chars'],
        truncation=True,
        is_split_into_words=True,
        max_length=512
    )
    
    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names
)
tokenized_val = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("토큰화 완료!")

데이터 로딩 중...
  - Train: 179553 samples
  - Val: 24938 samples
  - Test: 19951 samples

토큰화 중...


Map:   0%|          | 0/179553 [00:00<?, ? examples/s]

Map:   0%|          | 0/24938 [00:00<?, ? examples/s]

토큰화 완료!


In [8]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    true_predictions = []
    true_labels = []
    
    for prediction, label in zip(predictions, labels):
        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:
                true_predictions.append(pred_id)
                true_labels.append(label_id)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels, 
        true_predictions, 
        average='weighted',
        zero_division=0
    )
    
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [9]:
print("\n모델 학습 시작...")

Path("models")

training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("models/final_model")
tokenizer.save_pretrained("models/final_model")

print(f"\n학습 완료! 모델 저장: models/final_model")


모델 학습 시작...


C:\Users\dhkst\AppData\Local\Temp\ipykernel_18452\3999087097.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.330000,0.324001,0.886310,0.896017,0.888616
2,0.265000,0.273065,0.902674,0.907787,0.903903
3,0.204600,0.260626,0.908595,0.912654,0.910044



학습 완료! 모델 저장: models/final_model


In [2]:
tokenizer = AutoTokenizer.from_pretrained("models/final_model")
model = AutoModelForTokenClassification.from_pretrained(
    "models/final_model",
    use_safetensors=True
)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Device: {device}\n")

with open("processed/label_map.json", 'r', encoding='utf-8') as f:
    label_map = json.load(f)
id2label = {int(k): v for k, v in label_map['id2label'].items()}

Device: cuda



In [3]:
print("게임 용어 NER 추론")
print("=" * 60)
sentence = input("문장을 입력하세요: ")

if not sentence.strip():
    print("문장이 입력되지 않았습니다.")
else:
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)
        probabilities = torch.softmax(outputs.logits, dim=2)
    
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    predictions = predictions[0].cpu().numpy()
    probabilities = probabilities[0].cpu().numpy()
    
    entities = []
    current_entity = None
    
    for i, (token, pred_id) in enumerate(zip(tokens, predictions)):
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
        
        label = id2label[pred_id]
        confidence = probabilities[i][pred_id]
        
        if label.startswith('B-'):
            if current_entity:
                entities.append(current_entity)
            
            entity_type = label[2:]
            current_entity = {
                'term': token.replace('##', ''),
                'label': entity_type,
                'confidence': confidence
            }
        elif label.startswith('I-') and current_entity:
            entity_type = label[2:]
            if entity_type == current_entity['label']:
                current_entity['term'] += token.replace('##', '')
                current_entity['confidence'] = (current_entity['confidence'] + confidence) / 2
        else:
            if current_entity:
                entities.append(current_entity)
                current_entity = None
    
    if current_entity:
        entities.append(current_entity)
    
    print(f"\n입력: {sentence}")
    print(f"결과: {len(entities)}개 용어 인식")
    if entities:
        for entity in entities:
            print(f"  - {entity['term']} ({entity['label']}, {entity['confidence']:.2%})")
    else:
        print("  (인식된 게임 용어 없음)")

게임 용어 NER 추론


문장을 입력하세요:  탈진 도적이 아직도 날뛰다니..!



입력: 탈진 도적이 아직도 날뛰다니..!
결과: 1개 용어 인식
  - 탈진도적 (시스템용어, 52.50%)
